In [36]:
import pandas as pd
import datetime
import os
import numpy as np
import plotly.express as px

In [37]:
water_levels_common_id_df = pd.read_parquet('./data/raw/parquet/combined.parquet')
water_levels_common_id_df.rename(columns={'sourceDate': 'timestamp', 'value': 'water_level'}, inplace=True)
water_levels_common_id_df['is_outlier'] = None
water_levels_common_id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415631 entries, 0 to 415630
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   water_level  415631 non-null  float64            
 1   timestamp    415631 non-null  datetime64[ns, UTC]
 2   common_id    415631 non-null  object             
 3   is_outlier   0 non-null       object             
dtypes: datetime64[ns, UTC](1), float64(1), object(2)
memory usage: 12.7+ MB


In [38]:
outlier_stations_df = pd.read_csv('./data/stations.csv')
outlier_stations_df['outlier_timestamp'] = pd.to_datetime(outlier_stations_df['outlier_timestamp'], utc=True)
outlier_stations_df['start_timestamp'] = outlier_stations_df['outlier_timestamp'] - np.timedelta64(1, 'D')
outlier_stations_df['end_timestamp'] = outlier_stations_df['outlier_timestamp'] + np.timedelta64(1, 'D')
outlier_stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   water_name         26 non-null     object             
 1   station_name       26 non-null     object             
 2   outlier_timestamp  21 non-null     datetime64[ns, UTC]
 3   common_id          26 non-null     object             
 4   start_timestamp    21 non-null     datetime64[ns, UTC]
 5   end_timestamp      21 non-null     datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](3), object(3)
memory usage: 1.3+ KB


In [39]:
stations_df = outlier_stations_df.groupby('common_id').first().reset_index()[['common_id', 'water_name', 'station_name']]

stations_df

,common_id,water_name,station_name
0,2386-ch,Murg,Frauenfeld
1,2720050000-de,Sieg,Betzdorf
2,2736731000100-de,Wupper,Glüder
3,2756500000100-de,Angerbach,Ratingen
4,2824450000100-de,Vichtbach,Mulartshütte
5,36022-ie,Aghacashlaun,Aghacashlaun
6,39003-ie,Crana,Tullyarvan
7,42960105-de,Losse,Helsa
8,auto-1003803,Schmeie,Ebingen


In [40]:
water_levels_df = water_levels_common_id_df.merge(stations_df, on='common_id', how='left')
water_levels_df

,water_level,timestamp,common_id,is_outlier,water_name,station_name
0,19.5,2015-10-30 08:00:00+00:00,2824450000100-de,None,Vichtbach,Mulartshütte
1,19.1,2015-11-02 08:00:00+00:00,2824450000100-de,None,Vichtbach,Mulartshütte
2,19.1,2015-11-02 09:00:00+00:00,2824450000100-de,None,Vichtbach,Mulartshütte
3,19.1,2015-11-02 10:00:00+00:00,2824450000100-de,None,Vichtbach,Mulartshütte
4,19.1,2015-11-02 11:00:00+00:00,2824450000100-de,None,Vichtbach,Mulartshütte
...,...,...,...,...,...,...
415626,22.1,2022-04-02 12:15:00+00:00,36022-ie,None,Aghacashlaun,Aghacashlaun
415627,22.6,2022-04-02 12:45:00+00:00,36022-ie,None,Aghacashlaun,Aghacashlaun
415628,22.3,2022-04-02 13:15:00+00:00,36022-ie,None,Aghacashlaun,Aghacashlaun
415629,22.4,2022-04-02 13:45:00+00:00,36022-ie,None,Aghacashlaun,Aghacashlaun


In [41]:
outlier_stations_df

,water_name,station_name,outlier_timestamp,common_id,start_timestamp,end_timestamp
0,Vichtbach,Mulartshütte,NaT,2824450000100-de,NaT,NaT
1,Losse,Helsa,NaT,42960105-de,NaT,NaT
2,Murg,Frauenfeld,NaT,2386-ch,NaT,NaT
3,Wupper,Glüder,NaT,2736731000100-de,NaT,NaT
4,Angerbach,Ratingen,NaT,2756500000100-de,NaT,NaT
5,Crana,Tullyarvan,2020-07-27 16:00:00+00:00,39003-ie,2020-07-26 16:00:00+00:00,2020-07-28 16:00:00+00:00
6,Sieg,Betzdorf,2021-01-25 14:00:00+00:00,2720050000-de,2021-01-24 14:00:00+00:00,2021-01-26 14:00:00+00:00
7,Schmeie,Ebingen,2021-05-27 15:00:00+00:00,auto-1003803,2021-05-26 15:00:00+00:00,2021-05-28 15:00:00+00:00
8,Schmeie,Ebingen,2021-05-27 01:00:00+00:00,auto-1003803,2021-05-26 01:00:00+00:00,2021-05-28 01:00:00+00:00
9,Schmeie,Ebingen,2021-06-01 04:00:00+00:00,auto-1003803,2021-05-31 04:00:00+00:00,2021-06-02 04:00:00+00:00


In [69]:
outlier_values_lst = []
for idx, row in outlier_stations_df.dropna(subset=['outlier_timestamp']).iterrows():
    outlier_val_df = water_levels_df[(water_levels_df['common_id'] == row['common_id']) &
                                      (water_levels_df['timestamp'] >= row['start_timestamp']) &
                                      (water_levels_df['timestamp'] <= row['end_timestamp'])].copy()
    outlier_val_df['outlier_timestamp'] = row['outlier_timestamp']
    outlier_values_lst.append(outlier_val_df)

In [70]:
outlier_stations_df.groupby('common_id').count()


,water_name,station_name,outlier_timestamp,start_timestamp,end_timestamp
common_id,,,,,
2386-ch,1,1,0,0,0
2720050000-de,1,1,1,1,1
2736731000100-de,1,1,0,0,0
2756500000100-de,1,1,0,0,0
2824450000100-de,1,1,0,0,0
36022-ie,16,16,16,16,16
39003-ie,1,1,1,1,1
42960105-de,1,1,0,0,0
auto-1003803,3,3,3,3,3


In [71]:
outlier_values_lst[0]

,water_level,timestamp,common_id,is_outlier,water_name,station_name,outlier_timestamp
268745,62.1,2020-07-26 16:00:00+00:00,39003-ie,None,Crana,Tullyarvan,2020-07-27 16:00:00+00:00
268746,59.7,2020-07-26 17:00:00+00:00,39003-ie,None,Crana,Tullyarvan,2020-07-27 16:00:00+00:00
268747,58.3,2020-07-26 18:00:00+00:00,39003-ie,None,Crana,Tullyarvan,2020-07-27 16:00:00+00:00
268748,56.8,2020-07-26 19:00:00+00:00,39003-ie,None,Crana,Tullyarvan,2020-07-27 16:00:00+00:00
268749,55.3,2020-07-26 20:00:00+00:00,39003-ie,None,Crana,Tullyarvan,2020-07-27 16:00:00+00:00
268750,54.0,2020-07-26 21:00:00+00:00,39003-ie,None,Crana,Tullyarvan,2020-07-27 16:00:00+00:00
268751,52.9,2020-07-26 22:00:00+00:00,39003-ie,None,Crana,Tullyarvan,2020-07-27 16:00:00+00:00
268752,51.8,2020-07-26 23:00:00+00:00,39003-ie,None,Crana,Tullyarvan,2020-07-27 16:00:00+00:00
268753,50.8,2020-07-27 00:00:00+00:00,39003-ie,None,Crana,Tullyarvan,2020-07-27 16:00:00+00:00
268754,50.0,2020-07-27 01:00:00+00:00,39003-ie,None,Crana,Tullyarvan,2020-07-27 16:00:00+00:00


In [72]:
for idx, row in outlier_stations_df.dropna(subset=['outlier_timestamp']).iterrows():
    water_levels_df.loc[(water_levels_df['common_id'] == row['common_id']) & (water_levels_df['timestamp'] == row['outlier_timestamp'])]
    break

In [73]:
for outlier_df in outlier_values_lst:
    fig = px.line(outlier_df, x='timestamp', y='water_level', markers=True, title=f'{outlier_df.iloc[0]["water_name"]} - {outlier_df.iloc[0]["station_name"]} ({outlier_df.iloc[0]["common_id"]}) : {outlier_df.iloc[0]["outlier_timestamp"]}')
    fig.show()